# Barnacles
## Author: Arnav Singh

## Abstract

This notebook documents the process of developing a system to help scientists count barnacles in images more efficiently. The approach involves data preprocessing, exploration with basic models, and an attempt to implement a Vision Transformer (ViT) for improved accuracy.

---

## Table of Contents

1. [Introduction](#introduction)
2. [Data Loading and Preprocessing](#data-loading-and-preprocessing)
3. [Exploratory Data Analysis (EDA)](#exploratory-data-analysis-eda)
4. [Basic Model Exploration](#basic-model-exploration)
5. [Vision Transformer (ViT) Implementation](#vision-transformer-vit-implementation)
6. [Results and Analysis](#results-and-analysis)
7. [Conclusions](#conclusions)
8. [Learning and Next Steps](#learning-and-next-steps)

---

## Introduction

The goal is to automate the counting of barnacles within a fixed-size frame in images. This involves preprocessing the images, exploring basic models, and implementing a more advanced model like ViT to achieve higher accuracy.

## Data Loading and Preprocessing

In [ ]:
# Importing the necessasry libraries, make sure they are correctly installed if running the code on your local machine
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import tensorflow as tf

### Load Images and Masks
Next we load the images, and the masks for the data

In [ ]:
# Load images and masks
img1 = cv2.imread('Barnacles/img1.png')
mask1 = cv2.imread('Barnacles/mask1.png', 0)
img2 = cv2.imread('Barnacles/img2.png')
mask2 = cv2.imread('Barnacles/mask2.png', 0)